# The 8 data sets fit the models with tunned parameter

In [1]:
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

In [2]:
def XGB(datatrain, datavali, parameter_space,criteria='accuracy', randomstate=0):
    '''
    Combine the test data set and validation datase and use the tunned parameter
    '''
    from sklearn.model_selection import PredefinedSplit
    import numpy as np
    from xgboost.sklearn import XGBClassifier
    
    frames=pd.concat([datatrain, datavali])
    frames_reind=frames.set_index([pd.Index(range(len(frames)))])
    
    datadummy=pd.get_dummies(frames_reind)
    dataX=np.array(datadummy.drop('Default_ind', axis=1))
    datay=datadummy['Default_ind']
    
    XGB=XGBClassifier()
    
    from sklearn.model_selection import GridSearchCV
    
    modelGS=GridSearchCV(XGB, parameter_space,scoring=criteria)
    modelGS.fit(dataX, datay)
    
    return modelGS,dataX,datay

In [3]:
def Cate_to_object_excp_inq(dataset):
    return(dataset.astype({'non_mtg_acc_past_due_12_months_num': 'object','non_mtg_acc_past_due_6_months_num': 'object','mortgages_past_due_6_months_num': 'object','card_inq_24_month_num': 'float64','inq_12_month_num': 'float64','card_open_36_month_num': 'object','auto_open_36_month_num': 'object','ind_acc_XYZ': 'object'}))

In [4]:
def upsample(data):
    from sklearn.utils import resample
    
    df_majority=data[data.Default_ind==0]
    df_minority=data[data.Default_ind==1]
    largenumber=len(df_majority)
    df_minority_upsampled=resample(df_minority,replace=True,n_samples=int(1.2*largenumber),random_state=123)
    df_upsampled=pd.concat([pd.DataFrame(df_majority),pd.DataFrame(df_minority_upsampled)])
    return df_upsampled

## Listwise

In [5]:
list_train=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Listwise deletion/Listwise_deletion.csv")
list_test=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Listwise deletion/Listwise_deletion_Test.csv")
list_validation=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Listwise deletion/Listwise_deletion_validation.csv")

In [6]:
#Change to categorical
cat_list_train=Cate_to_object_excp_inq(list_train)
cat_list_test=Cate_to_object_excp_inq(list_test)
cat_list_validation=Cate_to_object_excp_inq(list_validation)

In [7]:
# Upsample train and validation
cat_list_up_train=upsample(cat_list_train)
cat_list_up_validation=upsample(cat_list_validation)

In [8]:
datadummy=pd.get_dummies(cat_list_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [9]:
parameter_space = {
    'colsample_bytree': [1.0],
     'gamma': [0],
     'learning_rate': [0.3],
     'max_depth': [5],
     'min_child_weight': [0.5],
     'n_estimators': [50],
     'nthread': [4],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5],
    'use_label_encoder':[False]
    }

In [10]:
clf_list,x_list,y_list=XGB(cat_list_up_train,cat_list_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [11]:
y_pred=clf_list.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3282,  528],
       [ 105,  236]])

In [12]:
clf_list.score(testX,testy)

0.8475066249096603

## Unconditional Median

In [13]:
median_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Unconditional median impute/Unconditional_median_impute.csv')
median_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Unconditional median impute/Unconditional_median_impute_Test.csv')
median_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Unconditional median impute/Unconditional_median_impute_Validation.csv')

In [14]:
#Change to categorical
cat_median_train=Cate_to_object_excp_inq(median_train)
cat_median_test=Cate_to_object_excp_inq(median_test)
cat_median_validation=Cate_to_object_excp_inq(median_validation)

In [15]:
# Upsample train and validation
cat_median_up_train=upsample(cat_median_train)
cat_median_up_validation=upsample(cat_median_validation)

In [16]:
datadummy=pd.get_dummies(cat_median_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [17]:
parameter_space = {
    'colsample_bytree': [0.8],
     'gamma': [0.5],
     'learning_rate': [0.3],
     'max_depth': [4],
     'min_child_weight': [0.5],
     'n_estimators': [30],
     'nthread': [4],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5],
    'use_label_encoder':[False]
    }

In [18]:
clf_median,x_median,y_median=XGB(cat_median_up_train,cat_median_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [19]:
y_pred=clf_median.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3737,  862],
       [  94,  307]])

In [20]:
clf_median.score(testX,testy)

0.8088

## Unconstrained Mean

In [21]:
umean_train=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Unconditional mean impute/Unconditional_mean_impute.csv")
umean_test=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Unconditional mean impute/Unconditional_mean_impute_Test.csv")
umean_validation=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Unconditional mean impute/Unconditional_mean_impute_Validation.csv")

In [22]:
umean_validation=umean_validation.drop(umean_validation.columns[0],axis=1)

In [23]:
#Change to categorical
cat_umean_train=Cate_to_object_excp_inq(umean_train)
cat_umean_test=Cate_to_object_excp_inq(umean_test)
cat_umean_validation=Cate_to_object_excp_inq(umean_validation)

In [24]:
# Upsample train and validation
cat_umean_up_train=upsample(cat_umean_train)
cat_umean_up_validation=upsample(cat_umean_validation)

In [25]:
datadummy=pd.get_dummies(cat_umean_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [26]:
parameter_space = {
    'colsample_bytree': [0.8],
     'gamma': [0.5],
     'learning_rate': [0.3],
     'max_depth': [4],
     'min_child_weight': [0.5],
     'n_estimators': [30],
     'nthread': [4],
     'objective': ['binary:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5]
    }

In [27]:
clf_umean,x_umean,y_umean=XGB(cat_umean_up_train,cat_umean_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [28]:
y_pred=clf_umean.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3735,  864],
       [  93,  308]])

In [29]:
clf_umean.score(testX,testy)

0.8086

## Random Forest

In [30]:
rf_train=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/rf impute/rf.csv")
rf_test=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/rf impute/rf_Test.csv")
rf_validation=pd.read_csv("/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/rf impute/rf_Vali.csv")

In [31]:
#Change to categorical
cat_rf_train=Cate_to_object_excp_inq(rf_train)
cat_rf_test=Cate_to_object_excp_inq(rf_test)
cat_rf_validation=Cate_to_object_excp_inq(rf_validation)

In [32]:
# Upsample train and validation
cat_rf_up_train=upsample(cat_rf_train)
cat_rf_up_validation=upsample(cat_rf_validation)

In [33]:
datadummy=pd.get_dummies(cat_rf_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [34]:
parameter_space = {
    'colsample_bytree': [1.0],
     'gamma': [1],
     'learning_rate': [0.3],
     'max_depth': [3],
     'min_child_weight': [1],
     'n_estimators': [40],
     'nthread': [4],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5],
    'use_label_encoder':[False]
    }

In [35]:
clf_rf,x_rf,y_rf=XGB(cat_rf_up_train,cat_rf_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [36]:
y_pred=clf_rf.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3695,  904],
       [  82,  319]])

In [37]:
clf_rf.score(testX,testy)

0.8028

## Cart

In [38]:
#Import dataset
cart_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Cart impute/Cart_impute.csv')
cart_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Cart impute/Cart_impute_Test.csv')
cart_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/Cart impute/Cart_impute_Vali.csv')

In [39]:
#Change to categorical
cat_cart_train=Cate_to_object_excp_inq(cart_train)
cat_cart_test=Cate_to_object_excp_inq(cart_test)
cat_cart_validation=Cate_to_object_excp_inq(cart_validation)

In [40]:
# Upsample train and validation
cat_cart_up_train=upsample(cat_cart_train)
cat_cart_up_validation=upsample(cat_cart_validation)

In [41]:
#test data Set
datadummy=pd.get_dummies(cat_cart_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [42]:
parameter_space = {
    'colsample_bytree': [0.9],
     'gamma': [0],
     'learning_rate': [0.3],
     'max_depth': [3],
     'min_child_weight': [0.5],
     'n_estimators': [40],
     'nthread': [4],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5],
    'use_label_encoder':[False]
    }

In [43]:
clf_cart,x_cart,y_cart=XGB(cat_cart_up_train,cat_cart_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [44]:
y_pred=clf_cart.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3696,  903],
       [  80,  321]])

In [45]:
clf_cart.score(testX,testy)

0.8034

## KNN

In [46]:
knn_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/imputed knn/KNN_impute.csv')
knn_train=knn_train.drop(knn_train.columns[0],axis=1)
knn_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/imputed knn/KNN_impute_test.csv')
knn_test=knn_test.drop(knn_test.columns[0],axis=1)
knn_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/imputed knn/KNN_impute_validation.csv')
knn_validation=knn_validation.drop(knn_validation.columns[0],axis=1)

In [47]:
cat_knn_train=Cate_to_object_excp_inq(knn_train)
cat_knn_test=Cate_to_object_excp_inq(knn_test)
cat_knn_validation=Cate_to_object_excp_inq(knn_validation)

In [48]:
cat_knn_up_train=upsample(cat_knn_train)
cat_knn_up_validation=upsample(cat_knn_validation)

In [49]:
datadummy=pd.get_dummies(cat_knn_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [50]:
parameter_space = {
    'colsample_bytree': [1.0],
     'gamma': [1],
     'learning_rate': [0.3],
     'max_depth': [4],
     'min_child_weight': [1],
     'n_estimators': [30],
     'nthread': [4],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5],
    'use_label_encoder':[False]
    }

In [51]:
clf_knn,x_knn,y_knn=XGB(cat_knn_up_train,cat_knn_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [52]:
y_pred=clf_knn.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3724,  875],
       [  84,  317]])

In [53]:
clf_knn.score(testX,testy)

0.8082

## EMB

In [54]:
EMB_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EMB_impute/EMB_impute.csv')
EMB_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EMB_impute/EMB_impute_Test.csv')
EMB_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EMB_impute/EMB_impute_Vali.csv')

In [55]:
#Change to categorical
cat_emb_train=Cate_to_object_excp_inq(EMB_train)
cat_emb_test=Cate_to_object_excp_inq(EMB_test)
cat_emb_validation=Cate_to_object_excp_inq(EMB_validation)

In [56]:
# Upsample train and validation
cat_emb_up_train=upsample(cat_emb_train)
cat_emb_up_validation=upsample(cat_emb_validation)

In [57]:
datadummy=pd.get_dummies(cat_emb_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [58]:
parameter_space = {
    'colsample_bytree': [1.0],
     'gamma': [1],
     'learning_rate': [0.3],
     'max_depth': [3],
     'min_child_weight': [0.5],
     'n_estimators': [40],
     'nthread': [4],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5],
    'use_label_encoder':[False]
    }

In [59]:
clf_emb,x_emb,y_emb=XGB(cat_emb_up_train,cat_emb_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [60]:
y_pred=clf_emb.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3688,  911],
       [  89,  312]])

In [61]:
clf_emb.score(testX,testy)

0.8

## EM

In [75]:
EM_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EM_imputed/Simulated_Data_Train_em.csv')
EM_train=EM_train.drop(EM_train.columns[0],axis=1)
EM_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EM_imputed/Simulated_Data_Test_em.csv')
EM_test=EM_test.drop(EM_test.columns[0],axis=1)
EM_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EM_imputed/Simulated_Data_validatiion_em.csv')
EM_validation=EM_validation.drop(EM_validation.columns[0],axis=1)

In [76]:
#Change to categorical
cat_em_train=Cate_to_object_excp_inq(EM_train)
cat_em_test=Cate_to_object_excp_inq(EM_test)
cat_em_validation=Cate_to_object_excp_inq(EM_validation)

In [77]:
# Upsample train and validation
cat_em_up_train=upsample(cat_em_train)
cat_em_up_validation=upsample(cat_em_validation)

In [78]:
datadummy=pd.get_dummies(cat_em_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [79]:
parameter_space = {
    'colsample_bytree': [1.0],
     'gamma': [1],
     'learning_rate': [0.3],
     'max_depth': [3],
     'min_child_weight': [0.5],
     'n_estimators': [50],
     'nthread': [4],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5],
    'use_label_encoder':[False]
    }

In [80]:
clf_em,x_em,y_em=XGB(cat_em_up_train,cat_em_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [81]:
y_pred=clf_emb.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix(testy,y_pred)

array([[3755,  844],
       [  93,  308]])

In [82]:
clf_em.score(testX,testy)

0.815